In [31]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np
import warnings
import csv
warnings. filterwarnings('ignore')

In [32]:
client = MongoClient()
trading = client['trading']


list_of_json = []
for el in trading.temp3.find():
    list_of_json.append(el)

df = pd.DataFrame(list_of_json)
df.drop(columns='_id', inplace=True)

seria_temp = df['url'].copy()
seria_temp.drop_duplicates(keep='first', inplace=True)
list_of_true_index = list(seria_temp.index)
df = df.loc[df.index[list_of_true_index]]

In [33]:
col_for_drop = df.columns[1:]
for col in df.columns[1:]:
    df[col]=[",".join(lists) for lists in df[col]]

col_for_split_p1 = ['total_revenue', 'gross_profit', 'net_income', 'pe', 'ebita']
for col in col_for_split_p1:
    list_of_name = [col+"_"+str(i) for i in range(1,9)] 
    df[list_of_name] = df[col].str.split(',',8, expand = True)

col_for_split_p2 = ['eps_predict', 'eps_true']
for col in col_for_split_p2:
    list_of_name = [col+"_"+str(i) for i in range(1,8)] 
    df[list_of_name] = df[col].str.split(',',7, expand = True)
    
    
df.drop(columns=col_for_drop, inplace=True)
df.drop_duplicates(inplace=True)


In [34]:
df.head(3)

,url,total_revenue_1,total_revenue_2,total_revenue_3,total_revenue_4,total_revenue_5,total_revenue_6,total_revenue_7,total_revenue_8,gross_profit_1,...,eps_predict_5,eps_predict_6,eps_predict_7,eps_true_1,eps_true_2,eps_true_3,eps_true_4,eps_true_5,eps_true_6,eps_true_7
0,https://ru.tradingview.com/symbols/NASDAQ-ADBE...,4.25B,4.35B,4.38B,4.45B,4.64B,4.81B,4.89B,18.79B,3.69B,...,3.50,3.68,3.79,3.20,3.37,3.35,3.40,3.60,3.80,3.91
1,https://ru.tradingview.com/symbols/NASDAQ-ADP/...,4.03B,4.51B,4.13B,4.22B,4.39B,4.93B,4.48B,18.01B,1.86B,...,1.79,1.93,2.46,1.65,1.65,2.21,1.50,1.86,1.96,2.52
2,https://ru.tradingview.com/symbols/NASDAQ-AKAM...,905.36M,903.65M,903.33M,881.90M,927.78M,915.70M,935.72M,3.66B,547.39M,...,1.23,1.26,1.32,1.45,1.49,1.39,1.35,1.26,1.37,1.40


In [9]:
col_for_num = ['total_revenue_1', 'total_revenue_2', 'total_revenue_3',
       'total_revenue_4', 'total_revenue_5', 'total_revenue_6',
       'total_revenue_7', 'total_revenue_8', 'gross_profit_1',
       'gross_profit_2', 'gross_profit_3', 'gross_profit_4', 'gross_profit_5',
       'gross_profit_6', 'gross_profit_7', 'gross_profit_8', 'net_income_1',
       'net_income_2', 'net_income_3', 'net_income_4', 'net_income_5',
       'net_income_6', 'net_income_7', 'net_income_8']

### Преобразования к числовому формату

In [35]:
def data_to_numeric(df):
    df = df.replace('', '0')
    for col_name in df.columns[1:]:
        for data in df[col_name]:
            if data != '':
                if data[-1]=='M':
                    data_dict_replace1 = {}
                    if data[0] != '−':
                        value = float(data[0:-1])*1e6
                    else:
                        value = -float(data[1:-1])*1e6
                    data_dict_replace1[data] = str(value)
                    df.replace(data_dict_replace1, inplace=True)
                    
    for col_name in df.columns[1:]:
        for data in df[col_name]:
            if data != '':
                if data[-1]=='B':
                    data_dict_replace1 = {}
                    if data[0] != '−':
                        value = float(data[0:-1])*1e9
                    else:
                        value = -float(data[1:-1])*1e9
                    data_dict_replace1[data] = str(value)
                    df.replace(data_dict_replace1, inplace=True)
                    
    for col_name in df.columns[1:]:
        for data in df[col_name]:
            if data != '':
                if data[-1]=='K':
                    data_dict_replace1 = {}
                    if data[0] != '−':
                        value = float(data[0:-1])*1e3
                    else:
                        value = -float(data[1:-1])*1e3
                    data_dict_replace1[data] = str(value)
                    df.replace(data_dict_replace1, inplace=True)              
    
                    
    for col_name in df.columns[1:]:
        for data in df[col_name]:
            if data != '':
                data_dict_replace1 = {}
                if data[0] == '−':
                    value = -float(data[1:-1])
                    data_dict_replace1[data] = str(value)
                    df.replace(data_dict_replace1, inplace=True) 
    
    
    for col in df.columns[1:]:
        df[col] = df[col].astype(float)    
    return df
        

In [38]:
df_n = data_to_numeric(df)
df_n.head(3)

,url,total_revenue_1,total_revenue_2,total_revenue_3,total_revenue_4,total_revenue_5,total_revenue_6,total_revenue_7,total_revenue_8,gross_profit_1,...,eps_predict_5,eps_predict_6,eps_predict_7,eps_true_1,eps_true_2,eps_true_3,eps_true_4,eps_true_5,eps_true_6,eps_true_7
0,https://ru.tradingview.com/symbols/NASDAQ-ADBE...,4.250000e+09,4.350000e+09,4.380000e+09,4.450000e+09,4.640000e+09,4.810000e+09,4.890000e+09,1.879000e+10,3.690000e+09,...,3.50,3.68,3.79,3.20,3.37,3.35,3.40,3.60,3.80,3.91
1,https://ru.tradingview.com/symbols/NASDAQ-ADP/...,4.030000e+09,4.510000e+09,4.130000e+09,4.220000e+09,4.390000e+09,4.930000e+09,4.480000e+09,1.801000e+10,1.860000e+09,...,1.79,1.93,2.46,1.65,1.65,2.21,1.50,1.86,1.96,2.52
2,https://ru.tradingview.com/symbols/NASDAQ-AKAM...,9.053600e+08,9.036500e+08,9.033300e+08,8.819000e+08,9.277800e+08,9.157000e+08,9.357200e+08,3.660000e+09,5.473900e+08,...,1.23,1.26,1.32,1.45,1.49,1.39,1.35,1.26,1.37,1.40


## Отбор позиций

### Растет выручка растут обороты

In [39]:
for el in df_n['url'].loc[(df['total_revenue_7'] > df['total_revenue_6']) & (df['gross_profit_7'] > df['gross_profit_6']) & (df['eps_true_7'] > df['eps_predict_7'])]:
    print(el)

https://ru.tradingview.com/symbols/NASDAQ-ADBE/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-AKAM/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-GOOGL/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-GOOG/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-ACN/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-ADM/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-MMM/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-ABT/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-ALK/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-AME/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-APH/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-AZO/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-AVY/financials-income-statement/
https://ru.tradingview.com/symbols/NYS

### PE падает профит растет current pe

In [43]:
for el in df['url'].loc[(df['eps_true_7'] > df['eps_predict_7'])& (df['pe_8'] < df['pe_7']) & (df['pe_8'] < df['pe_4']) & (df['pe_7'] < df['pe_3'])& (df['pe_7'] < df['pe_6']) & (df['total_revenue_7'] > df['total_revenue_6']) & (df['total_revenue_7'] > df['total_revenue_3']) & (df['gross_profit_7'] > df['gross_profit_6'])]:
    print(el)

https://ru.tradingview.com/symbols/NYSE-CLX/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-KO/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-CCI/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-EXR/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-F/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-GM/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-JCI/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-DXCM/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-EXPE/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-KHC/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-MCHP/financials-income-statement/
https://ru.tradingview.com/symbols/NASDAQ-PYPL/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-LMT/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-M

### PE падает профит растет квартальный

In [27]:
for el in df['url'].loc[(df['eps_true_7'] > df['eps_predict_7'])& (df['pe_7'] < df['pe_6']) & (df['pe_7'] < df['pe_3']) & (df['total_revenue_7'] > df['total_revenue_6']) & (df['gross_profit_7'] > df['gross_profit_6'])]:
    print(el)

https://ru.tradingview.com/symbols/NASDAQ-ADBE/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-ADM/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-MMM/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-BR/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-CLX/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-KO/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-CCI/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-EMR/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-EXR/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-FDS/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-FRT/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-F/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-GM/financials-income-statement/
https://ru.tradingview.com/symbols/NYSE-HAL/financial

### EPS выше ожиданий PE падает

In [28]:
df['url'].loc[(df['eps_true_7'] > df['eps_predict_7'])& (df['pe_7'] < df['pe_3']) & (df['pe_7'] < df['pe_6'])]

17     https://ru.tradingview.com/symbols/NYSE-AFL/fi...
18     https://ru.tradingview.com/symbols/NYSE-A/fina...
20     https://ru.tradingview.com/symbols/NYSE-ALK/fi...
25     https://ru.tradingview.com/symbols/NYSE-AEE/fi...
27     https://ru.tradingview.com/symbols/NYSE-AIG/fi...
                             ...                        
536    https://ru.tradingview.com/symbols/NYSE-ROK/fi...
537    https://ru.tradingview.com/symbols/NYSE-ROL/fi...
539    https://ru.tradingview.com/symbols/NYSE-CRM/fi...
556    https://ru.tradingview.com/symbols/NYSE-TGT/fi...
573    https://ru.tradingview.com/symbols/NYSE-UHS/fi...
Name: url, Length: 74, dtype: object

In [30]:
df_n.to_csv('trading_data_clared_09_2023.csv',index=False)